## Recipe-maker: Part 1  <img style="float: left;" src="assets/potatochef.png" width="150" height="150"> 

#### Introduction

In this notebook, I'm going to explore recipe data from _allrecipes.com_. It will be part 1 of the two parts I have split my project into:

* **Part 1**  
  
  My main question is 'What ingredients go together?'.
  
  But there are some problems with this question:  
  The data I am currently using includes all kinds of recipes: breads, cakes, desserts, pasta, soups etc. and it might be difficult and just extra work to divide the data, and without dividing the data, it might be difficult to answer the question well. However, the results might be more fun!
  
  Possible solutions:  
  * Divide the data in a way that would be easier to divide. E.g. use a subset of the data that only includes soup dishes.
  * Divide the data into a subset that includes single dishes cooked on the hob only. 
  * Tag the data with categories
  * Go back later and use a different dataset that I scrape myself to include only a subsection of _allrecipes.com_. 
  
  What will be the measure of how well ingredients go together?  
  **(I haven't figured this out yet)**
  * The ratings may be an indicator but ratings would reflect many other things, including how complicated the steps are, how long it takes to make the dish etc.  
  * The fact that a combination exists in the first place, or how frequently it exists, may be another indicator, but again there would be many factors contributing to a recipe's frequency.
  
  * A recipe involving cauliflower, chocolate and olives is less likely to make it on allrecipes and if it were, I'd think the reviews would be bad, so it might be an idea to create a combination of features to be the 'outcome'.  
  * Another possible solution: use a different source of data, for which recipes receive ratings for different factors seperately - I would be interested in a 'taste' rating.
  
  


* **Part 2**  
  
  My ultimate goal is to make a program that could do one of more or the following:  
  * Given some user-inputted ingredients, gives recommendation of other ingredients to use
  * Given a user-inputted list of ingredients, creates a recommended recipe for the user to follow (seems more difficult)

Please forgive me for my plan being a little half-baked. 

#### Imports

In [2]:
import pandas as pd
import numpy as np


#### Read the data

In [3]:
#recipes1 = pd.read_csv('data/recipes.csv')
recipes = pd.read_csv('data/recipes.csv', delimiter=';')
reviews = pd.read_csv('data/reviews.csv', delimiter=',')
recipes.shape

(12351, 10)

In [4]:
recipes.shape

(12351, 10)

In [5]:
reviews.shape

(1563566, 3)

In [6]:
recipes.head(10)

,Recipe Name,Review Count,Recipe Photo,Author,Prepare Time,Cook Time,Total Time,Ingredients,Directions,RecipeID
0,Golden Crescent Rolls Recipe,304,https://images.media-allrecipes.com/userphotos...,Mike A.,25 m,15 m,3 h 10 m,"yeast,water,white sugar,salt,egg,butter,flour,...","Dissolve yeast in warm water.**Stir in sugar, ...",7000
1,Poppy Seed Bread with Glaze Recipe,137,https://images.media-allrecipes.com/userphotos...,Christina Jun,15 m,1 h,1 h 20 m,"flour,salt,baking powder,poppy,butter,vegetabl...",'Preheat oven to 350 degrees F (175 degrees C)...,7001
2,Applesauce Bread I Recipe,124,https://images.media-allrecipes.com/userphotos...,GAF55,10 m,1 h 20 m,1 h 30 m,"flour,egg,white sugar,vegetable oil,applesauce...",Preheat oven to 350 degrees F (175 degrees C)....,7003
3,Apple Raisin Bread Recipe,39,https://images.media-allrecipes.com/userphotos...,Helen Hanson,15 m,1 h,1 h 15 m,"flour,baking powder,baking soda,salt,cinnamon,...",Preheat oven to 350 degrees F (175 degrees C)....,7006
4,Buttermilk Oatmeal Bread Recipe,41,https://images.media-allrecipes.com/userphotos...,Helen Hanson,10 m,1 h,1 h 40 m,"oat,buttermilk,vegetable oil,egg,brown sugar,f...",Mix oats with buttermilk. Let stand for 1/2 h...,7007
5,Kolaches II Recipe,27,https://images.media-allrecipes.com/userphotos...,Nan,30 m,20 m,2 h 5 m,"shortening,white sugar,salt,milk,egg,lemon,yea...",Cream shortening and sugar together. Stir in ...,7008
6,Whole Wheat Bread II Recipe,91,https://images.media-allrecipes.com/userphotos...,Ruth Uitto,20 m,35 m,2 h 25 m,"shortening,water,brown sugar,yeast,water,white...",Dissolve brown sugar in 1 cup hot water. Add ...,7009
7,Cottage Cheese Bread II Recipe,27,https://images.media-allrecipes.com/userphotos...,Kathy Nowell,5 m,3 h,3 h 5 m,"water,cottage cheese,vegetable oil,egg,flour,w...",Add the ingredients to the pan of your bread m...,7012
8,Mocha Chocolate Chip Banana Muffins Recipe,579,https://images.media-allrecipes.com/userphotos...,Shelley Ross,10 m,25 m,35 m,"margarine,white sugar,egg,banana,coffee,water,...",Preheat oven to 350 degrees F (175 degrees C)....,7013
9,'Ruby''s Special Cornbread Recipe ',4,https://images.media-allrecipes.com/userphotos...,Mitzi Lyons,10 m,45 m,55 m,"cornmeal,milk,egg,bell pepper,onion,garlic,sal...",Preheat oven to 350 degrees F (175 degrees C)....,7014


In [7]:
reviews.head(10)

,RecipeID,profileID,Rate
0,7000,675719,5.0
1,7000,1478626,5.0
2,7000,608663,5.0
3,7000,2785736,5.0
4,7000,594474,5.0
5,7000,5468,5.0
6,7000,2926455,5.0
7,7000,1896099,5.0
8,7000,25495,4.0
9,7000,539102,5.0
